<a href="https://colab.research.google.com/github/CodewithTanzeel/RAG_Vector_Embedding_Model/blob/main/RAG_EMBEDDING_MODEL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install -qU langchain-pinecone langchain-google-genai



In [ ]:
from google.colab import userdata
import getpass
import os
!pip uninstall -y pinecone pinecone-plugin-inference
!pip install -U "pinecone<6.0.0"
!pip install langchain-pinecone
from pinecone import Pinecone

pinecone_api_key = userdata.get("PINECONE_API_KEY")

pc = Pinecone(api_key=pinecone_api_key)


In [3]:
from pinecone import Pinecone, ServerlessSpec

index_name = "rag-pinecone-langchain"

existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]

if index_name not in existing_indexes:
    pc.create_index(
        name=index_name,
        dimension=768,  # the model we are using have 768 dimensions thats why we have used 768
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )

    while not pc.describe_index(index_name).status["ready"]:
        time.sleep(1)

index = pc.Index(index_name)


In [ ]:
index_stats = pc.describe_index("rag-pinecone-langchain")
print(index_stats)# checking stats of the created pinecone index!!!


In [5]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-004")#Declartion of chatmodelClass anad passing it an embedding model

In [ ]:
vector =embeddings.embed_query("We are building a RAG ")


In [ ]:
vector

In [ ]:
from langchain_pinecone import PineconeVectorStore
vector_store = PineconeVectorStore(index=index,embedding=embeddings)

In [ ]:
from langchain_core.documents import Document

document_1 = Document(
    page_content="I had chocalate chip pancakes and scrambled eggs for breakfast this morning.",
    metadata={"source": "tweet"},
)

In [ ]:
document_1

In [ ]:
# Data Saving procedure
from uuid import uuid4

from langchain_core.documents import Document

document_1 = Document(
    page_content="I had chocalate chip pancakes and scrambled eggs for breakfast this morning.",
    metadata={"source": "tweet"},
)

document_2 = Document(
    page_content="The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees.",
    metadata={"source": "news"},
)

document_3 = Document(
    page_content="Building an exciting new project with LangChain - come check it out!",
    metadata={"source": "tweet"},
)

document_4 = Document(
    page_content="Robbers broke into the city bank and stole $1 million in cash.",
    metadata={"source": "news"},
)

document_5 = Document(
    page_content="Wow! That was an amazing movie. I can't wait to see it again.",
    metadata={"source": "tweet"},
)

document_6 = Document(
    page_content="Is the new iPhone worth the price? Read this review to find out.",
    metadata={"source": "website"},
)

document_7 = Document(
    page_content="The top 10 soccer players in the world right now.",
    metadata={"source": "website"},
)

document_8 = Document(
    page_content="LangGraph is the best framework for building stateful, agentic applications!",
    metadata={"source": "tweet"},
)

document_9 = Document(
    page_content="The stock market is down 500 points today due to fears of a recession.",
    metadata={"source": "news"},
)

document_10 = Document(
    page_content="I have a bad feeling I am going to get deleted :(",
    metadata={"source": "tweet"},
)

documents = [
    document_1,
    document_2,
    document_3,
    document_4,
    document_5,
    document_6,
    document_7,
    document_8,
    document_9,
    document_10,
]


In [ ]:
len(documents)

In [ ]:

from uuid import uuid4

uuid4()


In [ ]:


uuids = [str(uuid4()) for _ in range(len(documents))]
vector_store.add_documents(documents=documents,ids=uuids)
#Delete items from vector store
#vector_store.delete(ids=[uuids[-1]])
# Data Retrieveing


In [ ]:
# Data Retrival
results = vector_store.similarity_search(
    "LangChain provides abstractions to make working with LLMs easy",
     k=2,
    filter={"source": "tweet"},
 )
for res in results:
    print(f"* {res.page_content} [{res.metadata}]")

In [ ]:
results = vector_store.similarity_search_with_score(
    "Will it be hot tomorrow?",
)
for res, score in results:
    print(f"* [SIM={score:3f}] {res.page_content} [{res.metadata}]")

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

In [ ]:
def answer_to_user(query:str):

    #Vector Search
    VectorResults = vector_store.similarity_search(query,k=2)
    #Pass to Model Vector Results + User Query
    final_answer = llm.invoke(f"Answer this user query{query},Here are some refrences{VectorResults}")
    return final_answer
    final_answer = ChatGoogleGenerativeAI(results,query)
    return final_answer



In [ ]:
answer = answer_to_user("LangChain provides abstractions to make working with LLMs easy")

In [ ]:
answer.content